In [51]:
import torch
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [52]:
class CustomDataset:
  def __init__(self, data, targets):
    self.data = data
    self.targets = targets
 
  def __len__(self):
    return self.data.shape[0]
  
  def __getitem__(self, idx):
    current_sample = self.data[idx, :]
    current_target = self.targets[idx]
    return {
        "x": torch.tensor(current_sample, dtype=torch.float),
        "y": torch.tensor(current_target, dtype=torch.long)
    }

In [53]:
data, targets = make_classification(n_samples=1000)
train_data, test_data, train_targets, test_targets = train_test_split(
    data, 
    targets, 
    stratify=targets
    )

In [54]:
train_dataset = CustomDataset(train_data,train_targets)
test_dataset = CustomDataset(test_data,test_targets)

In [55]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 4, num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 4, num_workers = 2)

In [56]:
model = lambda x, w, b: torch.matmul(x, w) + b

In [57]:
train_data.shape

(750, 20)

In [58]:
W = torch.randn(20, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

learning_rate = 0.001



In [59]:
# Calculate metric pre training

outputs = []
labels = []

with torch.no_grad():
  for data in test_loader:
    xtest = data["x"]
    ytest = data["y"]

    output = model(xtest, W, b)
    labels.append(ytest)
    outputs.append(output)

In [60]:
output_val_pre = torch.cat(outputs).view(-1)

In [61]:
label_val_pre = torch.cat(labels).view(-1)

In [62]:
metrics.roc_auc_score(label_val_pre, output_val_pre)

0.506432

In [63]:
for epoch in range(10):
  epoch_loss = 0
  counter = 0
  for data in train_loader:
    xtrain = data["x"]
    ytrain = data["y"]

    #print(W.grad)

    output = model(xtrain, W, b)
    loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
    epoch_loss = epoch_loss + loss.item()
    loss.backward()

    #print(W.grad)

    with torch.no_grad():
      W = W - learning_rate * W.grad
      b = b - learning_rate * b.grad
    #print(W.grad)
    
    W.requires_grad_(True)
    b.requires_grad_(True)
    counter += 1
    #break
  #break
  print(epoch, epoch_loss/counter)


0 13.74974182057888
1 6.604033913225569
2 3.2661498304694256
3 1.670795031724141
4 0.8948383561949781
5 0.5120413307041088
6 0.32092581809736154
7 0.22451254804717732
8 0.17542487103492022
9 0.15022550660007178


In [64]:
W, b

(tensor([[-0.0337],
         [-0.0113],
         [ 0.0066],
         [ 0.0342],
         [-0.0111],
         [ 0.0393],
         [-0.0195],
         [-0.0598],
         [ 0.0820],
         [-0.0470],
         [ 0.0502],
         [ 0.0042],
         [ 1.2151],
         [-0.0149],
         [ 0.0391],
         [-0.0210],
         [-0.0734],
         [-0.0163],
         [-0.5416],
         [-0.0216]], requires_grad=True),
 tensor([0.4402], requires_grad=True))

In [65]:
# Calculate metric post training

outputs = []
labels = []

with torch.no_grad():
  for data in test_loader:
    xtest = data["x"]
    ytest = data["y"]

    output = model(xtest, W, b)
    labels.append(ytest)
    outputs.append(output)

In [66]:
output_val_post = torch.cat(outputs).view(-1)

In [67]:
label_val_post = torch.cat(labels).view(-1)

In [68]:
metrics.roc_auc_score(label_val_post, output_val_post)

0.9115519999999999